In [1]:
import numpy as np
import open3d as o3d
import pandas as pd
from sklearn.cluster import KMeans
import random
import os
import glob
from sklearn.naive_bayes import BernoulliNB
from sklearn.cluster import KMeans

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
def preprocess_point_cloud(pcd):
    pcd = pcd.voxel_down_sample(voxel_size=0.02)

    radius_normal = 0.2
    print("estimating normals")
    pcd.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))

    radius_feature = 0.5
    print("computing fpfh")
    pcd_fpfh = o3d.pipelines.registration.compute_fpfh_feature(
        pcd,
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=100))
    fpfh = np.asarray(pcd_fpfh.data).T
    return pcd, fpfh

In [3]:
def cov_metrics(pcd):
    print("estimating covariances")
    covs = pcd.estimate_covariances()
    covs = np.asarray(pcd.covariances)

    print("computing eigen-based metrics")
    metrics = []
    for pt in covs:
        eigenvalues = np.linalg.eigvals(pt)
        e1, e2, e3 = eigenvalues
        
        linearity = (e1 - e2) / e1
        planarity = (e2 - e3) / e1
        scattering = e3 / e1
        omnivariance = (e1 * e2 * e3) ** (1 / 3)
        anisotropy = (e1 - e3) / e1
        eigentropy = -(e1 * np.log(e1) + e2 * np.log(e2) + e3 * np.log(e3))
        curvature = e3 / (e1 + e2 + e3)

        metrics.append((linearity, planarity, scattering, omnivariance, anisotropy, eigentropy, curvature))

    dtype = [('linearity', 'f8'), ('planarity', 'f8'), ('scattering', 'f8'), 
            ('omnivariance', 'f8'), ('anisotropy', 'f8'), ('eigentropy', 'f8'), 
            ('curvature', 'f8')]
    
    metrics_array = np.array(metrics, dtype=dtype)
  
    return np.array([tuple(row) for row in metrics_array])

In [4]:
def get_clusters(df):
    df['cluster'] = KMeans(n_clusters=4,
                           random_state=0,
                           init=initial_centroids).fit(df).labels_
    return df

In [ ]:
print("reading point cloud")
pcd = o3d.io.read_point_cloud("C:/Users/ellie/OneDrive/Desktop/lidar_local/ccb-3_preprocessed.pcd")
print("preprocessing point cloud")
pcd, fpfh = preprocess_point_cloud(pcd)
print("getting metrics")
cov = cov_metrics(pcd)

cov_headers = ['linearity', 'planarity', 'scattering', 'omnivariance', 'anisotropy', 'eigentropy', 'curvature']
header = ['x', 'y', 'z'] + [f'feature{i}' for i in range(fpfh.shape[1])] + cov_headers
all_metrics = np.hstack([np.asarray(pcd.points), fpfh, cov])

df = pd.DataFrame(all_metrics, columns=header)

reading point cloud


In [ ]:
cov_headers = ['linearity', 'planarity', 'scattering', 'omnivariance', 'anisotropy', 'eigentropy', 'curvature']
header = ['x', 'y', 'z'] + [f'feature{i}' for i in range(fpfh.shape[1])] + cov_headers
all_metrics = np.hstack([np.asarray(pcd.points), fpfh, cov])
df = pd.DataFrame(all_metrics, columns=header)

In [ ]:
df = df[df['z'] <= 0.25]
df['cluster'] = KMeans(n_clusters=3, random_state=0).fit(df).labels_
print(df)

In [ ]:
from open3d.web_visualizer import draw

just_clusters = df[["x", "y", "z", "cluster"]]
cluster_0 = just_clusters[just_clusters['cluster'] == 0]
cluster_1 = just_clusters[just_clusters['cluster'] == 1]
cluster_2 = just_clusters[just_clusters['cluster'] == 2]
cluster_3 = just_clusters[just_clusters['cluster'] == 3]
cluster_4 = just_clusters[just_clusters['cluster'] == 4]
cluster_5 = just_clusters[just_clusters['cluster'] == 5]
cluster_6 = just_clusters[just_clusters['cluster'] == 6]
cluster_7 = just_clusters[just_clusters['cluster'] == 7]
cluster_8 = just_clusters[just_clusters['cluster'] == 8]
cluster_9 = just_clusters[just_clusters['cluster'] == 9]
cluster_10 = just_clusters[just_clusters['cluster'] == 10]

zero = cluster_0[['x', 'y', 'z']].values
one = cluster_1[['x', 'y', 'z']].values
two = cluster_2[['x', 'y', 'z']].values
three = cluster_3[['x', 'y', 'z']].values
four = cluster_4[['x', 'y', 'z']].values
five = cluster_5[['x', 'y', 'z']].values
six = cluster_6[['x', 'y', 'z']].values
seven = cluster_7[['x', 'y', 'z']].values
eight = cluster_8[['x', 'y', 'z']].values
nine = cluster_9[['x', 'y', 'z']].values
ten = cluster_10[['x', 'y', 'z']].values


pcd0 = o3d.geometry.PointCloud()
pcd0.points = o3d.utility.Vector3dVector(zero)
draw(pcd0)

pcd1 = o3d.geometry.PointCloud()
pcd1.points = o3d.utility.Vector3dVector(one)
draw(pcd1)

pcd2 = o3d.geometry.PointCloud()
pcd2.points = o3d.utility.Vector3dVector(two)
draw(pcd2)

pcd3 = o3d.geometry.PointCloud()
pcd3.points = o3d.utility.Vector3dVector(three)
draw(pcd3)

pcd4 = o3d.geometry.PointCloud()
pcd4.points = o3d.utility.Vector3dVector(four)
draw(pcd4)

pcd5 = o3d.geometry.PointCloud()
pcd5.points = o3d.utility.Vector3dVector(five)
draw(pcd5)

pcd6 = o3d.geometry.PointCloud()
pcd6.points = o3d.utility.Vector3dVector(six)
draw(pcd6)

pcd7 = o3d.geometry.PointCloud()
pcd7.points = o3d.utility.Vector3dVector(seven)
draw(pcd7)

pcd8 = o3d.geometry.PointCloud()
pcd8.points = o3d.utility.Vector3dVector(eight)
draw(pcd8)

pcd9 = o3d.geometry.PointCloud()
pcd9.points = o3d.utility.Vector3dVector(nine)
draw(pcd9)

pcd10 = o3d.geometry.PointCloud()
pcd10.points = o3d.utility.Vector3dVector(ten)
draw(pcd10)